In [1]:
import os
import os.path
import datetime as dt

from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

In [5]:
SCOPES = ["https://www.googleapis.com/auth/calendar"]

class GoogleCalendarManager:
    def __init__(self):
        self.service = self._authenticate()

    def _authenticate(self):
        creds = None

        if os.path.exists("token.json"):
            creds = Credentials.from_authorized_user_file("token.json", SCOPES)

        if not creds or not creds.valid:
            if creds and creds.expired and creds.refresh_token:
                creds.refresh(Request())
            else:
                flow = InstalledAppFlow.from_client_secrets_file(r"C:\Users\capor\Downloads\Desafio\client_secret_293424317647-v6d3sj0uin8uk9qb372ap3l2i2iu5d0m.apps.googleusercontent.com.json", SCOPES)
                creds = flow.run_local_server(port=0)

            # Save the credentials for the next run
            with open("token.json", "w") as token:
                token.write(creds.to_json())

        return build("calendar", "v3", credentials=creds)

    def list_upcoming_events(self, max_results=10):
        now = dt.datetime.utcnow().isoformat() + "Z"
        tomorrow = (dt.datetime.now() + dt.timedelta(days=5)).replace(hour=23, minute=59, second=0, microsecond=0).isoformat() + "Z"

        events_result = self.service.events().list(
            calendarId='primary', timeMin=now, timeMax=tomorrow,
            maxResults=max_results, singleEvents=True,
            orderBy='startTime'
        ).execute()
        events = events_result.get('items', [])

        if not events:
            print('No upcoming events found.')
        else:
            for event in events:
                start = event['start'].get('dateTime', event['start'].get('date'))
                print(start, event['summary'],event['id'])
        
        return events

    def create_event(self, summary, start_time, end_time, timezone, attendees=None):
        event = {
            'summary': summary,
            'start': {
                'dateTime': start_time,
                'timeZone': timezone,
            },
            'end': {
                'dateTime': end_time,
                'timeZone': timezone,
            }
        }

        if attendees:
            event["attendees"] = [{"email": email} for email in attendees]

        try:
            event = self.service.events().insert(calendarId="primary", body=event).execute()
            print(f"Event created: {event.get('htmlLink')}")
        except HttpError as error:
            print(f"An error has occurred: {error}")

    def update_event(self, event_id, summary=None, start_time=None, end_time=None):
        event = self.calendar_service.events().get(calendarId='primary', eventId=event_id).execute()

        if summary:
            event['summary'] = summary

        if start_time:
            event['start']['dateTime'] = start_time.strftime('%Y-%m-%dT%H:%M:%S')

        if end_time:
            event['end']['dateTime'] = end_time.strftime('%Y-%m-%dT%H:%M:%S')

        updated_event = self.calendar_service.events().update(
            calendarId='primary', eventId=event_id, body=event).execute()
        return updated_event

    def delete_event(self, event_id):
        self.calendar_service.events().delete(calendarId='primary', eventId=event_id).execute()
        return True
    

calendar = GoogleCalendarManager()

calendar.list_upcoming_events()

calendar.create_event("Hola Martitaaaa Martitaaaa","2024-06-23T19:30:00+02:00","2024-06-23T21:30:00+02:00","Europe/Madrid",["martaeserre97@gmail.com","capopo2005@gmail.com", "caporga@gmail.com", "yepa@gmail.com"])

2024-06-23T19:30:00+02:00 Hola Martita Martita 2709nrhip1luep8m19heoidd5o
2024-06-23T19:30:00+02:00 Hola Martita Martita o52noo42uoag0fi5474dui43oc
Event created: https://www.google.com/calendar/event?eid=bHEyNGk3aTNwdmtnOWxlbG5xNGduOGVyZXMgY2Fwb3JnYUBt


In [10]:
calendar.create_event("Hola Martita Martita Martita","2024-06-20T19:30:00+02:00","2024-06-20T21:30:00+02:00","Europe/Madrid",["martaeserre97@gmail.com","capopo2005@gmail.com"])

Event created: https://www.google.com/calendar/event?eid=NzhqYWRiN2IyMHFiYXVsN2ZvcWtpa3ZkbG8gY2Fwb3JnYUBt
